# TME 3.1 : Greedy Profile Motif Search 

Numéro étudiant 1 : 21241759<br>
Numéro étudiant 2 : 22106550 <br>

## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm Greedy Profile Motif Search.


1\. Nous allons réutiliser les fonctions du precedent pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [12]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Même chose hein
    """
    sequence = "".join([random.choice(nuc) for _ in range(n)])
    if upper:
        return sequence
    return sequence.lower()

def modifierMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)

    nbPos_real = min(nbpos, len(motif))
    allPos = list(range(len(motif)))

    for _ in range(nbPos_real):
        index = random.choice(range(len(allPos)))
        changeIndex = allPos[index]
        del allPos[index]

        nv_L = generateRandomSequence(1, upper)
        motifM[changeIndex] = nv_L
    return "".join(motifM)        


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    
    sequences = []

    motif = generateRandomSequence(k)
    print(motif)
    motif_new = motif
    for _ in range(t):
        base_seq = generateRandomSequence(n - k)
        if random.random() < f:
            motif_new = modifierMotif(motif, v)
        sequences.append(insertMotif(base_seq, motif_new, random.choice(range(len(base_seq)))))
        motif_new = motif
    
    return sequences

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence
f= 0.5 #frequence de variation du motif.

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

gcctt
['cgccttgtat', 'cgtggccttc', 'gagccttaac']
['CGCCTTGTAT', 'CGTGGCCTTC', 'GAGCCTTAAC']


2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [13]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
	"""
	Génère un vecteur de position aléatoires
	entrée sequences: matrice de dimension txn contenant les sequences
	entrée k: taille du motif
	sortie s: vecteur de position aléatoires, une position par séquence
  REMARQUE les positions doivent être inférieur à n-k, ou n est la taille de sequence
	"""
	s = []
	for i in sequences:
		s.append(random.randint(0, (len(i)-k)-1))
	return s

s = generateRandomS(adn, k)
print (s)

[2, 0, 4]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profile (matrice de fréquence). 


In [14]:
#extract sequences from seqs
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn contenant les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs de taille k extrait des séquences
    """
    motifs = []    
    for i in range(len(seqs)):
        motifs.append("".join(seqs[i][s[i]:s[i]+k]).upper())

    return motifs
        
motifs = extractSeqs(s, adn, k)
print (motifs)


#Construire un profile = matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    q = len(nuc)
    PWM = np.zeros((q, k))
    for i in motifs:
        for j in range(k):
            if i[j] == "A":
                PWM[0][j] += 1
            if i[j] == "C":
                PWM[1][j] += 1
            if i[j] == "G":
                PWM[2][j] += 1
            if i[j] == "T":
                PWM[3][j] += 1
  
    return PWM

MF = profile(motifs, k, nuc)
print(MF)

['CCTTG', 'CGTGG', 'CTTAA']
[[0. 0. 0. 1. 1.]
 [3. 1. 0. 0. 0.]
 [0. 1. 0. 1. 2.]
 [0. 1. 3. 1. 0.]]


4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes.

In [15]:
def generatePWM(MF):
  """
  Transforme la matrice de fréquence en PWM
  entrée MF: matrice de fréquence
  sortie PWM: matrice de probabilité (poids positions)
  """
  PWM = MF.copy()
  for i in range(len(MF[0])):
    summ = 0
    for j in range(len(MF)):
      summ += MF[j][i]
    for j in range(len(MF)):
      PWM[j][i] /= summ
    
  return PWM

PWM = generatePWM(MF)
print(PWM)

[[0.         0.         0.         0.33333333 0.33333333]
 [1.         0.33333333 0.         0.         0.        ]
 [0.         0.33333333 0.         0.33333333 0.66666667]
 [0.         0.33333333 1.         0.33333333 0.        ]]


5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [16]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    prob = []

    for i in range(len(motif)):
        match motif[i]:
            case "A":
                prob.append(PWM[0][i])
            case "C":
                prob.append(PWM[1][i])
            case "G":
                prob.append(PWM[2][i])
            case "T":
                prob.append(PWM[3][i])
            case _:
                print("nucléotide n'est pas accepté")
  
    return sum(prob)/len(prob)

prob = probability(PWM, "CGCTA")
print (prob)

0.39999999999999997


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [17]:
from itertools import product

def trouverLowComplexe(motif):
    """
    Eleve les motifs peu complexe ayant 
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    sortie motifsClean: dictionnaire de motifs sans les motifs peu complexe.
    """
    twoLetterCombs = ["".join(tup) for tup in list(product(["A", "T", "G", "C"], repeat=2))]
    for i in motifs:
        mot = i.upper()
        if mot.count("A") > 4 or mot.count("T") > 4 or mot.count("G") > 4 or mot.count("C") > 4:
            return True
        if True in [x*3 in motif for x in twoLetterCombs]:
            return True
        
    return False

def pMostProbkmer(PWM, k, sequence):
    """
    calcul la position du k-mer le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: la position la plus probable 
    """
    maxProb = 0
    s = 0 # position du k-mer le plus probable dans la sequence
    
    for i in range(0, len(sequence)-k, 1):
        if probability(PWM, sequence[i:i+k]) > maxProb:
            if trouverLowComplexe(sequence[i:i+k]):
                continue
            maxProb = probability(PWM, sequence[i:i+k])
            s = i
    return s

s = pMostProbkmer(PWM, k, "GCTATCGCTA")
print(s)

0


7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiens les k-mer le plus probables.

In [18]:
def getNewS(PWM, k, sequences):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: vecteur avec les positions les plus problables
    """

    s = []
    for i in sequences:
        tmp = pMostProbkmer(PWM, k, i)
        s.append(tmp)
   
    return s

ns = getNewS(PWM, k, adn)
print (ns)

[2, 0, 3]


8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre, 
faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [19]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréquence
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    return P1 != P2

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    for i in range(k):
        sc += max(MF[0][i], MF[1][i], MF[2][i], MF[3][i])
    return sc

ns = getNewS(PWM, k, adn)
print (s, ns)

sc = getScore(MF, k)
print (sc)

0 [2, 0, 3]
10.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 
BONUS: Pour ameliorer la performance vous pouvez abandonner les positions qui génèrent des motifs peu complexes.

In [20]:
def GreedyProfileMotifSearch(sequences, k):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
	  entrée sequences: matrice de dimension txn contenant les séquences 
	  entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    """
    nuc = ('A', 'C', 'G', 'T')
    
    s = generateRandomS(sequences, k)
    motifs = extractSeqs(s, sequences, k)
    MF = profile(motifs, k, nuc)
    PWM = generatePWM(MF)
    
    bestScore = 0
    
    s = getNewS(PWM, k, sequences)
    # Optimiser dans getNewS
    # trouverLowComplexes doit verifier si on n'a pas choisi des motifs peu complexes, maos il faut l'amiliorer, car la compléxité depend de la longueur 
    for i, j in zip(sequences, s):
        score = 0
        for l in range(k):
            match i[j + l]:
                case "A":
                    score += MF[0][l]
                case "C":
                    score += MF[1][l]
                case "G":
                    score += MF[2][l]
                case "T":
                    score += MF[3][l]
        if score > bestScore:
            bestScore = score
    
    return s, bestScore

print (adn)
s, bestScore = GreedyProfileMotifSearch(adn, k)   
motifs = extractSeqs(s, adn, k)
print (motifs)
        

['CGCCTTGTAT', 'CGTGGCCTTC', 'GAGCCTTAAC']
['CCTTG', 'GCCTT', 'CCTTA']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse: Oui, après quelques éxécutions on trouve le morif de taille k implanté.

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

In [21]:
def GreedyProfileMotifSearchIte(sequences, k, It):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
	  entrée séquences: matrice de dimension txn contenant les séquences
	  entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= sequence consensus du motif
    """
    pos = {}
    consensus = {}
   
    nuc = ('A', 'C', 'G', 'T')
    s = generateRandomS(sequences, k)
    for i in range(It):
        s = generateRandomS(sequences, k)
        motifs = extractSeqs(s, sequences, k)
        MF = profile(motifs, k, nuc)
        PWM = generatePWM(MF)
        
        bestScore = 0
        
        s = getNewS(PWM, k, sequences)
        for m, j in zip(sequences, s):
            # if trouverLowComplexe(m[j:j+k]): #Test pour les sequences peu complexes, je crois que ça marche pas
            #     continue
            # else:
                score = 0
                for l in range(k):
                    match m[j + l]:
                        case "A":
                            score += MF[0][l]
                        case "C":
                            score += MF[1][l]
                        case "G":
                            score += MF[2][l]
                        case "T":
                            score += MF[3][l]
                if score > bestScore:
                    bestScore = score
                    if (bestScore in pos.keys()): 
                        pos[bestScore].add(j)
                    else:
                        pos[bestScore] = set()
                        pos[bestScore].add(j)
                    if (bestScore in consensus.keys()):
                        consensus[bestScore].add(m[j:j+k])
                    else:
                        consensus[bestScore] = set()
                        consensus[bestScore].add(m[j:j+k])

    return dict(sorted(pos.items(), reverse=True)), dict(sorted(consensus.items(), reverse=True))


s, consensus = GreedyProfileMotifSearchIte(adn, k, 50)

print (s)
print (consensus)

{15.0: {1}, 12.0: {1, 2, 3}, 11.0: {0, 1}, 10.0: {0, 1, 2}, 9.0: {0, 1, 2, 3, 4}, 8.0: {0, 1, 2, 3, 4}, 7.0: {1, 3, 4}, 6.0: {0}, 5.0: {0}}
{15.0: {'GCCTT'}, 12.0: {'GCCTT', 'CCTTG', 'GGCCT'}, 11.0: {'GCCTT', 'CGCCT'}, 10.0: {'GCCTT', 'CGCCT', 'TGGCC'}, 9.0: {'CGTGG', 'CCTTA', 'TTGTA', 'CGCCT', 'GCCTT', 'CCTTG', 'GGCCT', 'GTGGC'}, 8.0: {'TTGTA', 'CGCCT', 'GCCTT', 'CCTTG', 'CTTGT'}, 7.0: {'GCCTT', 'TTGTA', 'CTTGT'}, 6.0: {'CGCCT'}, 5.0: {'CGCCT'}}


In [22]:
#Affiche les sequence consensus de top motifs
def printTopFMotifsScore(consensus, top):

    motifsSort = sorted(consensus.keys())
    motifsSort.reverse()
    for i in range(top):
      print (consensus[motifsSort[i]])

printTopFMotifsScore(consensus, 2)




{'GCCTT'}
{'GCCTT', 'CCTTG', 'GGCCT'}


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [23]:

def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

In [26]:
k=8

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_3.fasta"

sequencesChip = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequencesChip = sequencesChip + revSequences
s, consensus = GreedyProfileMotifSearchIte(sequencesChip, k, 20)
printTopFMotifsScore(consensus, 10)

s, consensus


CTGATCAGCA 4616 8
{'AAATAATT'}
{'AATTAATT'}
{'AAATAATA'}
{'AAATAATT'}
{'AAATTATT'}
{'TAAATATT'}
{'AAATTATT'}
{'AAATTAAT'}
{'TAAATATT'}
{'TAATAATT'}


({22274.0: {1},
  22214.0: {1},
  22200.0: {0},
  22195.0: {1},
  22176.0: {1},
  22166.0: {1},
  22164.0: {1},
  22159.0: {1},
  22148.0: {1},
  22143.0: {0},
  22119.0: {1},
  22118.0: {1},
  22106.0: {1},
  22100.0: {1},
  22096.0: {0},
  22094.0: {0},
  22093.0: {0},
  22089.0: {0},
  22085.0: {0},
  22084.0: {1},
  22079.0: {1},
  22077.0: {1},
  22068.0: {1},
  22067.0: {1},
  22064.0: {0},
  22063.0: {1},
  22059.0: {1},
  22054.0: {0},
  22053.0: {1},
  22052.0: {0},
  22047.0: {1},
  22044.0: {1},
  22042.0: {0},
  22039.0: {1},
  22030.0: {0, 1},
  22026.0: {1},
  22023.0: {1},
  22021.0: {1},
  22015.0: {1},
  22013.0: {0},
  22011.0: {1},
  22007.0: {1},
  22002.0: {0},
  22001.0: {1},
  22000.0: {1},
  21997.0: {1},
  21996.0: {0},
  21993.0: {0},
  21989.0: {0},
  21985.0: {1},
  21984.0: {1},
  21981.0: {1},
  21980.0: {1},
  21969.0: {1},
  21966.0: {0},
  21965.0: {1},
  21961.0: {1},
  21959.0: {1},
  21955.0: {1},
  21953.0: {0},
  21952.0: {0},
  21947.0: {0},
  219